# Plotting routine for the June2 2022 Event - PSP

In [ ]:
test = 'test'

In [8]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocess as mp
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### setting the times for py3DCORE

In [21]:
t_launch = datetime.datetime(2022, 9, 5, 16, tzinfo=datetime.timezone.utc) # 

t_s = datetime.datetime(2022, 9, 5, 20, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 9, 6, 2, 30, tzinfo=datetime.timezone.utc)

t_fit = [
        # datetime.datetime(2022, 9, 5, 18, tzinfo=datetime.timezone.utc),
       # datetime.datetime(2022, 9, 5, 19, 5, tzinfo=datetime.timezone.utc),
       # datetime.datetime(2022, 9, 5, 20, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 5, 21, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 5, 22, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 6, 0, 5, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 6, 2, 5, tzinfo=datetime.timezone.utc)
     ]


In [22]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['09/05/2022, 21:00:00', '09/05/2022, 22:00:00', '09/06/2022, 00:05:00', '09/06/2022, 02:05:00']


#### loading the pickle files

In [23]:
path = 'output/psp05092022_512_RTN_1/'

filepath = fp.loadpickle(path, number=-1)

In [24]:
filepath[:-7]

'output/psp05092022_512_RTN_1/07'

#### extracting parameters

In [25]:
res_mineps, res_mean, res_std, ind, keys, allres = fp.get_params(filepath, give_mineps=True)

print(ind)

2023-06-07 17:40:49.849 INFO    py3dcore.fluxplot: Retrieved the following parameters for the run with minimum epsilon:
2023-06-07 17:40:49.850 INFO    py3dcore.fluxplot:  --cme_longitude -55.91
2023-06-07 17:40:49.851 INFO    py3dcore.fluxplot:  --cme_latitude -3.05
2023-06-07 17:40:49.872 INFO    py3dcore.fluxplot:  --cme_inclination 17.01
2023-06-07 17:40:49.873 INFO    py3dcore.fluxplot:  --cme_diameter_1au 0.55
2023-06-07 17:40:49.874 INFO    py3dcore.fluxplot:  --cme_aspect_ratio 3.82
2023-06-07 17:40:49.878 INFO    py3dcore.fluxplot:  --cme_launch_radius 8.41
2023-06-07 17:40:49.881 INFO    py3dcore.fluxplot:  --cme_launch_velocity 1422.34
2023-06-07 17:40:49.882 INFO    py3dcore.fluxplot:  --t_factor 93.08
2023-06-07 17:40:49.893 INFO    py3dcore.fluxplot:  --cme_expansion_rate 1.14
2023-06-07 17:40:49.897 INFO    py3dcore.fluxplot:  --magnetic_decay_rate 1.64
2023-06-07 17:40:49.900 INFO    py3dcore.fluxplot:  --magnetic_field_strength_1au 8.19
2023-06-07 17:40:49.908 INFO    

Results :

	cme_longitude: 		-38.86 +/- 13.62
	cme_latitude: 		-34.25 +/- 14.07
	cme_inclination: 		34.35 +/- 11.92
	cme_diameter_1au: 		0.54 +/- 0.09
	cme_aspect_ratio: 		3.58 +/- 0.92
	cme_launch_radius: 		8.42 +/- 2.28
	cme_launch_velocity: 		1198.05 +/- 329.37
	t_factor: 		-23.64 +/- 90.66
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.99 +/- 1.01
	background_drag: 		2.60 +/- 0.85
	background_velocity: 		285.90 +/- 150.95
8


In [26]:
out = fp.get_ensemble_stats(filepath)
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,expansion rate,B decay rate,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.00000,512.000000,512.000000,512.000000
mean,-38.859707,-34.248028,34.349968,0.543847,3.578821,8.421638,1198.053467,-23.638605,1.140000,1.64000,6.989943,2.597457,285.901794
std,13.631161,14.084848,11.935607,0.086003,0.924060,2.283221,329.694031,90.753128,0.000008,0.00001,1.009015,0.853495,151.100098
min,-63.767471,-66.743965,11.214437,0.294679,1.005341,5.000923,700.226318,-249.771774,1.140000,1.64000,5.010398,0.313853,50.089344
25%,-49.150329,-44.307841,25.898751,0.482004,2.937446,6.604059,928.226929,-82.963509,1.140000,1.64000,6.258531,1.995622,166.950203
50%,-40.645077,-35.446672,33.113714,0.545850,3.620324,8.049504,1153.224976,-21.677762,1.140000,1.64000,6.886184,2.650096,265.461899
75%,-30.905410,-26.013303,41.349020,0.600452,4.369005,9.824703,1420.792145,32.820446,1.140000,1.64000,7.676630,3.320461,385.490379
max,15.275434,14.144319,72.727646,0.774221,4.993153,14.929168,1977.268799,233.557587,1.140000,1.64000,10.030303,3.967475,688.069946


In [27]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [28]:
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    f.write('Summary of results')
    f.write('\n\n')
    f.write('Results:')
    f.write('\n')
    for i in range(1, len(keys)):    
        f.write("\t{}: \t\t{:.02f} +/- {:.02f}".format(keys[i], res_mean[i], res_std[i]))
        f.write('\n')
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for i in range(1, len(keys)):
        f.write(" --{} {:.2f}".format(keys[i], res_mineps[i]))
        f.write('\n')
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')

#### loading and plotting the fitting results

In [29]:
t_s = datetime.datetime(2022, 9, 5, 18, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 9, 6, 4, tzinfo=datetime.timezone.utc)

In [30]:
fp.fullinsitu(observer='PSP', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='SPP_RTN', best=False, 
              ensemble=True, save_fig=True, legend=True, max_index=512, title=False, fit_points=True)

2023-06-07 17:41:26.251 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


2022-09-05 18:00:00+00:00 2022-09-06 04:00:00+00:00


In [ ]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

In [ ]:
print(res_mineps[1])

#### Plotting the fluxrope and spacecraft positions

In [17]:
#measurement times 
#tm0 =  datetime.datetime(2022, 6, 2, 6)
tm1 =  t_launch + datetime.timedelta(hours=40)
#tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo'], planetlist=['Earth'], t=tm1, filepath=filepath, legend=False, view_azim=res_mineps[1]-90, title=False,
                view_elev=0, view_radius=0.7, index=0)

Results :

	cme_longitude: 		122.40 +/- 5.74
	cme_latitude: 		-1.52 +/- 8.33
	cme_inclination: 		20.55 +/- 47.98
	cme_diameter_1au: 		0.30 +/- 0.02
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53
	cme_launch_velocity: 		1405.48 +/- 328.54
	t_factor: 		161.74 +/- 45.97
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_drag: 		1.62 +/- 0.72
	background_velocity: 		613.52 +/- 56.56


2023-06-07 13:29:56.742 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-07 10:00:00+00:00


### make fluxrope 3d movie

In [18]:
def make_frame(k):
    tm0 = t_launch
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['psp', 'solo'], planetlist=['Earth'], legend=False, title=False,
              t=t, filepath=filepath, save_fig=True, view_radius=0.7, view_elev=90)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/' + frmstr + '.jpg', dpi=300)  
    return fig
    

In [19]:
inn = [i for i in range(100)]


mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[60:100])
mpool.close()
mpool.join()

print('done 4/4')

Results :
Results :


	cme_longitude: 		122.40 +/- 5.74
	cme_longitude: 		122.40 +/- 5.74	cme_latitude: 		-1.52 +/- 8.33

	cme_inclination: 		20.55 +/- 47.98	cme_latitude: 		-1.52 +/- 8.33

	cme_inclination: 		20.55 +/- 47.98	cme_diameter_1au: 		0.30 +/- 0.02

	cme_diameter_1au: 		0.30 +/- 0.02	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53

	cme_launch_radius: 		14.97 +/- 0.53	cme_launch_velocity: 		1405.48 +/- 328.54
	cme_launch_velocity: 		1405.48 +/- 328.54

	t_factor: 		161.74 +/- 45.97	t_factor: 		161.74 +/- 45.97

	cme_expansion_rate: 		1.14 +/- 0.00	cme_expansion_rate: 		1.14 +/- 0.00

	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00	magnetic_field_strength_1au: 		6.44 +/- 0.69

	background_drag: 		1.62 +/- 0.72	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_velocity: 		613.52 +/- 56.56

	background_drag: 		1.62 +/- 0.72
	background_velocity: 		613.52 +/- 56.56


2023-06-07 13:32:53.554 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-05 21:00:00+00:00
Results :

	cme_longitude: 		122.40 +/- 5.74


2023-06-07 13:32:53.628 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


	cme_latitude: 		-1.52 +/- 8.33
t_snap: 	cme_inclination: 		20.55 +/- 47.982022-09-05 18:00:00+00:00

	cme_diameter_1au: 		0.30 +/- 0.02
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53
	cme_launch_velocity: 		1405.48 +/- 328.54
	t_factor: 		161.74 +/- 45.97
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00Results :


	magnetic_field_strength_1au: 		6.44 +/- 0.69	cme_longitude: 		122.40 +/- 5.74

	cme_latitude: 		-1.52 +/- 8.33	background_drag: 		1.62 +/- 0.72

	cme_inclination: 		20.55 +/- 47.98	background_velocity: 		613.52 +/- 56.56
	cme_diameter_1au: 		0.30 +/- 0.02

	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53
	cme_launch_velocity: 		1405.48 +/- 328.54
	t_factor: 		161.74 +/- 45.97
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_drag: 		1.62 +/- 0.72
	background_velocity: 		613.52 +/- 56.56


...

2023-06-07 13:33:06.963 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-06 13:00:00+00:00
observer: Parker Solar Probe


2023-06-07 13:33:06.976 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-06 13:00:00+00:00


SpiceNOTSUPPORTED: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(NOTSUPPORTED) --

Unexpected SPK type 19 subtype -73811712 found in type 19 segment within mini-segment 1.

spkpos_c --> SPKPOS --> SPKEZP --> SPKGPS --> SPKPVN --> SPKR19

================================================================================

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
fig = fp.full3d_multiview(tm1, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

